In [1]:
import sys
import numpy as np
import pandas as pd
import cv2
from pycocotools.coco import COCO
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage

class Process(object):
    numKeypoint = 17
    keypointsName = ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 
                    'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 
                    'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 
                    'right_knee', 'left_ankle', 'right_ankle']

    skeleton = [[15, 13], [13, 11], [16, 14], 
                [14, 12], [11, 12], [5, 11], 
                [6, 12], [5, 6], [5, 7], [6, 8], 
                [7, 9], [8, 10], [1, 2], [0, 1], 
                [0, 2], [1, 3], [2, 4], [3, 5], [4, 6]]
    
    trainAnnoPath = 'COCO/annotations/person_keypoints_train2017.json'
    valAnnoPath = 'COCO/annotations/person_keypoints_val2017.json'
    
    augmentation = iaa.Resize(256)
    

    def loadData(self, setType):
        if setType == 'train':
            coco = COCO(self.trainAnnoPath)
        elif setType == 'val':
            coco = COCO(self.valAnnoPath)
            
        Data = []
        for aid in coco.anns.keys():
            ann =  coco.anns[aid]
            keypoints = ann['keypoints']
            
            if  keypoints.count(0) == 0: 
                imageName = 'COCO/' + setType + '2017/' + coco.imgs[ann['image_id']]['file_name']

                if (ann['image_id'] not in coco.imgs) or ann['iscrowd'] or (np.sum(keypoints[2::3]) == 0) or (ann['num_keypoints'] == 0):
                    continue

                data = dict(image_id = ann['image_id'], imgpath = imageName, bbox=ann['bbox'], keypoints=keypoints)
                Data.append(data)
            else:
                continue
            
        return Data
    
    def imagesToNdarry(self, data): 
        imgPath = []
        i = 0
        for i in range(len(data)):
            img = cv2.imread(data[i]['imgpath'])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgPath.append(img)
            
        return np.array(imgPath)
    
    def splitKeypoints(self, data):
        kpsList = []
        visibiltyList = []
        for ann in data:
            keypoints = np.array(ann['keypoints']).astype(int)
            keypoints =  keypoints.reshape(17,3)
            
            keypoints = self.reshapeKeypoints(keypoints)
            kpsList.append(keypoints['kps'])
            visibiltyList.append(keypoints['visibility'])

        return kpsList, visibiltyList
    
    
    def reshapeKeypoints(self, keypoints):
        keypoint = []
        visibility = []

        for kps in keypoints: 
            x,y,v = kps 
            keypoint.append(Keypoint(x,y))
            visibility.append(v)
            
        res = dict()
        res['kps'] = keypoint; res['visibility'] = visibility
        return res
    
    def augmentKeypoints(self, keypoints, shape):
        augKpsList = []
        for kps in keypoints:
            augKps = self.augmentation.augment_keypoints(KeypointsOnImage(kps, shape))
            augKpsList.append(augKps)
        return augKpsList
    
    def mergeKpsVisibility(self, augKps, visibility, kpShape, MergeShape):
        kpsList = [] 
        i = 0
        for i in range(len(augKps)):
            for keypoint in augKps[i]:
                x,y = keypoint.x, keypoint.y
                kpsList.append(x)
                kpsList.append(y)
        kpsList = np.array(kpsList).reshape(kpShape)
        merge = np.dstack((kpsList, visibility))
        merge = merge.reshape(MergeShape)
        return merge
    
    def augmentationOnAll(self, images, kps):
        seq = iaa.Sequential([
            iaa.Resize(112),            
        ])

        imgAugList = []
        kpsAugList = []
        
        i = 0 
        for i in range(len(images)):
            imgAug, kpsAug = seq(image=images[i], keypoints=KeypointsOnImage(kps[i],shape=images[i].shape))
            imgAugList.append(imgAug)
            kpsAugList.append(kpsAug)
            
        return np.array(imgAugList), kpsAugList
    
    def formatToDataframe(self, anno ,keypoints):
        col_name  = ['nose_x', 'nose_y', 'nose_v',
                     'left_eye_x','left_eye_y', 'left_eye_v', 
                     'right_eye_x', 'right_eye_y', 'right_eye_v',
                     'left_ear_x', 'left_ear_y', 'left_ear_v',
                     'right_ear_x', 'right_ear_y', 'right_ear_v',
                    'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_v',
                     'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_v',
                     'left_elbow_x', 'left_elbow_y', 'left_elbow_v',
                     'right_elbow_x', 'right_elbow_y', 'right_elbow_v', 
                    'left_wrist_x', 'left_wrist_y', 'left_wrist_v',
                     'right_wrist_x', 'right_wrist_y', 'right_wrist_v',
                     'left_hip_x', 'left_hip_y', 'left_hip_v', 
                     'right_hip_x', 'right_hip_y', 'right_hip_v',
                     'left_knee_x', 'left_knee_y', 'left_knee_v',
                    'right_knee_x', 'right_knee_y', 'right_knee_v', 
                     'left_ankle_x', 'left_ankle_y', 'left_ankle_v', 
                     'right_ankle_x', 'right_ankle_y', 'right_ankle_v',
                     'image_path'  
                    ]
        data = []

        i = 0
        for ann, kps in zip(anno, keypoints):
            cur_row = []
#                 bbox = np.array(anno[i]['bbox']).astype(int)
#             keypoints = np.array(anno[i]['keypoints']).astype(int)
#                 for loc in bbox:
#                     cur_row.append(loc)
            for key in kps:
                cur_row.append(int(key))
            cur_row.append(anno[i]['imgpath'])
            data.append(cur_row)

        return pd.DataFrame(data=data, columns=col_name)


processing = Process()

In [2]:
from sklearn.model_selection import train_test_split as tts
import numpy as np


class Setup(object):
    setType = ''
    batchsize = 0
    fetchedData = []
    
    def __init__ (self, setType, batchSize):
        self.setType = setType
        self.batchsize = batchSize
        
    def getTrainingSet(self):
        batch = processing.loadData('train')
        batch = batch[:self.batchsize]
        
        images = processing.imagesToNdarry(batch)
        
        kpsList, visibilityList = processing.splitKeypoints(batch)
        
        x, kps = processing.augmentationOnAll(images, kpsList)
        Y = processing.mergeKpsVisibility(kps, visibilityList, (self.batchsize, 17,2), (self.batchsize, 51))
        images = [] 
        return x, Y, batch

In [10]:
setup = Setup('train', 8000)
x , Y, batch = setup.getTrainingSet()

loading annotations into memory...
Done (t=10.31s)
creating index...
index created!


In [11]:
# for i in Y: 
#     for j in i:
#         print(j)
#         print('\n')

In [12]:
print(batch[0])
print(Y[0])

{'image_id': 528962, 'imgpath': 'COCO/train2017/000000528962.jpg', 'bbox': [369.94, 98.6, 69.95, 208.14], 'keypoints': [407, 124, 2, 410, 117, 2, 402, 119, 2, 419, 119, 2, 399, 122, 2, 427, 143, 2, 388, 143, 2, 433, 168, 2, 378, 170, 2, 431, 183, 2, 380, 187, 2, 416, 195, 2, 392, 194, 2, 421, 245, 2, 385, 214, 2, 418, 289, 2, 391, 247, 2]}
[91.16799927 37.03466797  2.         91.83999634 34.94400024  2.
 90.04800415 35.54133606  2.         93.85600281 35.54133606  2.
 89.37599945 36.43733215  2.         95.647995   42.70933151  2.
 86.91200256 42.70933151  2.         96.99199677 50.1760025   2.
 84.67199707 50.7733345   2.         96.54399872 54.65600204  2.
 85.11999512 55.85066605  2.         93.18400574 58.23999786  2.
 87.80799866 57.94133377  2.         94.30400085 73.17333221  2.
 86.23999786 63.91466522  2.         93.63200378 86.31466675  2.
 87.58399963 73.77066803  2.        ]


In [13]:
df = processing.formatToDataframe(batch, Y )

In [14]:
df.to_csv('augCOCO.csv',encoding='utf-8', index=False)